## base64解码

3个字节(24位)一组， 每6位一组编码

24位一组， 每8位(1字节)一组解码

In [1]:
e = 'YWJjZA=='

In [2]:
table = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/'

In [3]:
table.index('Y')

24

In [4]:
table.index('W')

22

In [5]:
table.index('J')

9

In [6]:
table.index('j')

35

In [9]:
tmp = (35 << 0) + (9 << 6) + (22 << 12) + (24 << 18)

In [17]:
tmp.to_bytes(3, 'little')

b'cba'

In [15]:
def b64decode(data:str) -> bytes:
    table = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/'
    decoded = bytearray()
    s = 0
    for e in range(4, len(data)+1, 4):
        tmp = 0
        for i, c in enumerate(data[s:e]):
            if c != '=':
                tmp += table.index(c) << 24 - (i+1) * 6
            else:
                tmp += 0 << 24 - (i+1) * 6
        decoded.extend(tmp.to_bytes(3, 'big'))
        s += 4
    return bytes(decoded.rstrip(b'\x00'))

In [16]:
b64decode(e)

b'abcd'

## 带过期的LRU缓存装饰器

LRU

* FIFO： 先进先出
* LFU: 最近最少使用      计数
* LRU：最近最久未使用    计时

value里加一个最后一次使用的时间戳

In [20]:
import inspect
import functools
import datetime


def cache(maxsize=128, expire=0):
    def make_key(fn, args, kwargs):
        ret = []
        names = set()
        params = inspect.signature(fn).parameters
        keys = list(params.keys())
        for i, arg in enumerate(args):
            ret.append((keys[i], arg))
            names.add(keys[i])
        ret.extend(kwargs.items())
        names.update(kwargs.keys())
        for k, v in params.items():
            if k not in names:
                ret.append((k, v.default))
        ret.sort(key=lambda x: x[0])
        return '&'.join(['{}={}'.format(name, arg) for name, arg in ret])
    
    def _cache(fn):
        data = {}
        @functools.wraps(fn)
        def wrap(*args, **kwargs):
            key = make_key(fn, args, kwargs)
            now = datetime.datetime.now().timestamp()
            if key in data.keys():
                value, timestamp, _ = data[key]
                if expire == 0 or now - timestamp < expire:
                    data[key] = (value, timestamp, now)
                    return value
                else:
                    data.pop(key)
            value = fn(*args, **kwargs)
            if len(data) >= maxsize: 
                # 过期清理
                if expire != 0:
                    expires = set()
                    for k, (_, timestamp, _) in data.items():
                        if now - timestamp >= expire:
                            expires.add(k)
                    for k in expires:
                        data.pop(k)
            if len(data) >= maxsize:
                # 换出
                k = sorted(data.items(), key=lambda x: x[1][2])[0][0]
                data.pop(k)
            data[key] = (value, now, now)
            return value
        return wrap
    return _cache

In [22]:
import inspect
import functools
import datetime


def cache(maxsize=128, expire=0):
    def make_key(fn, args, kwargs):
        ret = []
        names = set()
        params = inspect.signature(fn).parameters
        keys = list(params.keys())
        for i, arg in enumerate(args):
            ret.append((keys[i], arg))
            names.add(keys[i])
        ret.extend(kwargs.items())
        names.update(kwargs.keys())
        for k, v in params.items():
            if k not in names:
                ret.append((k, v.default))
        ret.sort(key=lambda x: x[0])
        return '&'.join(['{}={}'.format(name, arg) for name, arg in ret])
    
    def _cache(fn):
        data = {}
        queue = []
        @functools.wraps(fn)
        def wrap(*args, **kwargs):
            key = make_key(fn, args, kwargs)
            now = datetime.datetime.now().timestamp()
            if key in data.keys():
                value, timestamp = data[key]
                queue.remove(key)
                if expire == 0 or now - timestamp < expire:
                    queue.insert(0, key)
                    return value
                else:
                    data.pop(key)
            value = fn(*args, **kwargs)
            if len(data) >= maxsize: 
                # 过期清理
                if expire != 0:
                    expires = set()
                    for k, (_, timestamp, _) in data.items():
                        if now - timestamp >= expire:
                            expires.add(k)
                    for k in expires:
                        queue.remove(k)
                        data.pop(k)
            if len(data) >= maxsize:
                # 换出
                #k = sorted(data.items(), key=lambda x: x[1][2])[0][0]
                k = queue.pop()
                data.pop(k)
            data[key] = (value, now)
            queue.insert(0, key)
            return value
        return wrap
    return _cache

In [23]:
from collections import namedtuple

In [42]:
Node = {'data': '', 'prev': '', 'next': ''}

In [62]:
Item = namedtuple('Item', ['key', 'value', 'timestamp'])

In [53]:
def linked_list():
    _head = None
    _tail = None
    
    def put(item):
        nonlocal _head
        nonlocal _tail
        if _head is None:
            _head = [item, None, None]
        else:
            node = [item, None, _head]
            _head[1] = node
            _head = node
        if _tail is None:
            _tail = _head
        return _head
    
    def pop():
        nonlocal _tail
        if _tail is None:
            _head = None
            return None
        node = _tail
        _tail = node[1]
        return node
    
    def remove(node):
        nonlocal _head
        nonlocal _tail
        if node is _head:
            _head = node[2]
        if node is _tail:
            pop()
            return
        node[1][2] = node[2]
        node[2][1] = node[1]
    
    return put, pop, remove
        
        

In [64]:
def linked_list():
    _head = None
    _tail = None
    
    def put(item):
        nonlocal _head
        nonlocal _tail
        if _head is None:
            _head = {'data': item, 'prev': None, 'next': None}
        else:
            node = {'data': item, 'prev': None, 'next': _head}
            _head['prev'] = node
            _head = node
        if _tail is None:
            _tail = _head
        return _head
    
    def pop():
        nonlocal _tail
        if _tail is None:
            _head = None
            return None
        node = _tail
        _tail = node['prev']
        return node
    
    def remove(node):
        nonlocal _head
        nonlocal _tail
        if node is _head:
            _head = node['next']
        if node is _tail:
            pop()
            return
        node['prev']['next'] = node['next']
        node['next']['prev'] = node['prev']
    
    return put, pop, remove

In [65]:
import inspect
import functools
import datetime


def cache(maxsize=128, expire=0):
    def make_key(fn, args, kwargs):
        ret = []
        names = set()
        params = inspect.signature(fn).parameters
        keys = list(params.keys())
        for i, arg in enumerate(args):
            ret.append((keys[i], arg))
            names.add(keys[i])
        ret.extend(kwargs.items())
        names.update(kwargs.keys())
        for k, v in params.items():
            if k not in names:
                ret.append((k, v.default))
        ret.sort(key=lambda x: x[0])
        return '&'.join(['{}={}'.format(name, arg) for name, arg in ret])
    
    def _cache(fn):
        data = {}
        put, pop, remove = linked_list()
        @functools.wraps(fn)
        def wrap(*args, **kwargs):
            key = make_key(fn, args, kwargs)
            now = datetime.datetime.now().timestamp()
            if key in data.keys():
                node = data[key]
                item = node['data']
                remove(node)
                if expire == 0 or now - item.timestamp < expire:
                    data[key] = put(item)
                    return value
                else:
                    data.pop(key)
            value = fn(*args, **kwargs)
            if len(data) >= maxsize: 
                # 过期清理
                if expire != 0:
                    expires = set()
                    for k, node in data.items():
                        if now - node['data'].timestamp >= expire:
                            pop(node)
                            expires.add(k)
                    for k in expires:
                        data.pop(k)
            if len(data) >= maxsize:
                # 换出
                #k = sorted(data.items(), key=lambda x: x[1][2])[0][0]
                node = pop()
                data.pop(node['data'].key)
            node = put(Item(key, value, now))
            data[key] = node
            return value
        return wrap
    return _cache

## 命令分发器

In [98]:
import inspect
from collections import namedtuple


def dispatcher(default_handler=None):
    Handler = namedtuple('Handler', ['fn', 'params'])
    
    commands = {}
    if default_handler is None:
        default_handler = lambda *args, **kwargs: print('not found')
    
    def register(command):
        def _register(fn):
            params = inspect.signature(fn).parameters
            commands[command] = Handler(fn, params)
            return fn
        return _register
    
    def run():
        while True:
            command, _, params = input('>>').partition(':')
            # command:x,y,z=1
            if command.strip() == 'quit':
                return
            handler = commands.get(command.strip(), Handler(default_handler, {}))
            
            args = []
            kwargs = {}
            param_values = list(handler.params.values())
            for i, param in enumerate(params.split(',')):
                if '=' in param:
                    name, _, value = param.partition('=')
                    p = handler.params.get(name.strip())
                    if p is not None and p.annotation != inspect.Parameter.empty:
                        kwargs[name.strip()] = p.annotation(value)
                    else:
                        kwargs[name.strip()] = value
                else:
                    if len(param_values) > i and  param_values[i].annotation != inspect.Parameter.empty:
                        args.append(param_values[i].annotation(param.strip()))
                    else:
                        args.append(param.strip())
            ret = handler.fn(*args, **kwargs)
            if ret is not None:
                print(ret)
    
    return register, run
            

In [99]:
register, run = dispatcher()

In [100]:
@register('add')
def add(x: int, y:int):
    return x + y

In [101]:
run()

>>add:x=3,y=5
8
>>add:3,y=5
8
>>quit


## 优先队列

In [102]:
queue = []

In [118]:
def queue(): # O(n)
    lst = []
    
    def put(value, p): # O(n)
        for i, item in enumerate(lst):
            if item[1] <= p:
                lst.insert(i, (value, p))
                break
        else:
            lst.append((value, p))
    
    def pop(): # O(1)
        if lst:
            return lst.pop()[0]
        return None
    
    return put, pop

In [111]:
put, pop = queue() 

In [114]:
put(2, 3)

In [115]:
pop()

5

In [116]:
pop()

2

In [117]:
pop()

3

heap

* heap 是完全二叉树
* 父节点总是比子节点大(小)

* 完全二叉树可以使用列表表示
* data[0] 是根节点
* data[i].left = data[2i+1]
* data[i].right = data[2i+2]
* data[i].parent = data[floor((i-1)/2)]

In [140]:
def heap():
    data = []
    
    def put(value, p):
        i = len(data)
        data.append((value, p))
        pi = (i-1) // 2
        while pi >= 0 and p > data[pi][1]:
            data[i], data[pi] = data[pi], data[i]
            i = pi
            pi = (i-1) // 2
    
    def pop():
        if not data:
            return None
        if len(data) == 1:
            return data[0][0]
        i = 0
        ret = data[i][0]
        data[i] = data.pop()
        li = 2 * i + 1
        ri = 2 * i + 2
        while data and li < len(data):
            ci = li
            if ri < len(data) and data[ri][1] > data[li][1]:
                ci = ri
            if data[ci][1] > data[i][1]:
                data[ci], data[i] = data[i], data[ci]
                i = ci
                li = 2 * i + 1
                ri = 2 * i + 2
            else:
                break
        return ret
    
    return put, pop

In [127]:
put, pop = heap()

In [128]:
put(0, 35)

In [129]:
put(0, 28)

In [130]:
put(0, 78)

In [133]:
pop()

0

In [134]:
import random

In [135]:
random.randint(0, 100)

4

In [137]:
lst = [random.randint(0, 100) for x in range(20)]

In [138]:
lst

[85, 29, 90, 13, 75, 49, 97, 92, 15, 79, 63, 54, 20, 78, 53, 7, 18, 1, 92, 54]

In [139]:
for x in lst:
    put(0, x)

for _ in lst:
    print(pop()[1])

97
92
92
90
85
79
78
75
63
54
54
53
49
29
28
20
18
15
13
7
